In [1]:
%matplotlib inline
import pandas as pd
from pandas.plotting import scatter_matrix
from sklearn.tree import DecisionTreeClassifier,export_graphviz
from sklearn.model_selection import train_test_split
from sklearn import metrics
import numpy as np

df_rede = pd.read_csv("Dados-Dados.csv", encoding = "UTF-8", sep = ",")
#Somente as colunas que interessam para a classificação
df_rede = df_rede[['age','Medu','Fedu','traveltime','studytime','failures','famrel','freetime','goout','Dalc','Walc','health','absences','G1','G2','G3']]


In [2]:
df_rede.head()

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,18,4,4,2,2,0,4,3,4,1,1,3,6,5,6,6
1,17,1,1,1,2,0,5,3,3,1,1,3,4,5,5,6
2,15,1,1,1,2,3,4,3,2,2,3,3,10,7,8,10
3,15,4,2,1,3,0,3,2,2,1,1,5,2,15,14,15
4,16,3,3,1,2,0,4,3,2,1,2,5,4,6,10,10


In [4]:
def build_tree(data, labels, tree, depth = 1):
    classes, counts = np.unique(labels, return_counts=True)
    n_classes = classes.shape[0]

    # criétio de parada
    if not stopping_criterion(n_classes, depth, tree.max_depth):
        node = Node()

        # encontra melhor ponto de corte dado a região atual do espaço
        # de acordo com critério de impureza escolhido
        feature, threshold = find_cut_point(data, labels, 
                                            tree.impurity_criterion)
        
        # aplicando o limiar para particionar o espaço
        mask = data[:, feature] <= threshold
        
        # contruindo árvore recursivamente para
        # os sub-espaço da direita e da esquerda.
        left = build_tree(data[mask], labels[mask], tree, depth + 1)
        right = build_tree(data[~mask], labels[~mask], tree, depth + 1)
     
        return Node(feature=feature, threshold=threshold, left=left, right=right)

    # calcula a quantidade de exemplos por classe nesse nó folha
    # e instancia um nó folha com essas quantidades, lembre-se que isso
    # será usado para predição. 
    values = np.zeros(tree.n_classes)
    values[classes] = counts
    return Node(is_leaf=True, counts=values)

In [5]:
def find_cut_point(data, labels, impurity_criterion = gini_criterion):
  """ find the best cut point 
  
  Parameters
  ----------
  data: numpy array-like = [n_samples, n_features]
  labels: numpy array-like, shape = [n_samples]
  impurity_criterion: callable, default=gini_criterion
  
  Return
  ------
  feature, threshold
  """
  n_samples, n_features = data.shape

  max_info_gain = np.iinfo(np.int32).min
  feat_id = 0
  best_threshold = 0

  # pré-calculando a impureza da região atual
  H_parent = impurity_criterion(data, labels)
  # para cada um dos atributos
  # vamos tentar encontrar o limiar que maximiza o ganho de informação
  for j in range(n_features):
    # só nos interessa os valores ordenados únicos 
    # do atributo j nessa região do espaço
    values = np.unique(data[:, j])
    
    for i in range(values.shape[0] - 1):
      # usamos o ponto médio dos valores possíveis
      # como limiar candidato para o ponto de corte
      threshold = (values[i] + values[i + 1]) / 2.

      mask = data[:, j] <= threshold

      info_gain = H_parent \
                  - (mask.sum() * impurity_criterion(data[mask], labels[mask]) \
                  + (~mask).sum() * impurity_criterion(data[~mask], labels[~mask])) \
                  / float(n_samples)

      if max_info_gain < info_gain:
        best_threshold = threshold
        feat_id = j
        max_info_gain = info_gain
        
  return feat_id, best_threshold

NameError: name 'gini_criterion' is not defined